In [ ]:
pip install tensorflow

  Using cached tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached grpcio-1.66.1-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metada

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install necessary packages (only needed in the first run, then can be skipped)
!pip install python-chess anytree loguru PyYAML

In [1]:
# Imports
import chess
import chess.engine
import torch
import torch.optim as optim
import torch.nn as nn
from loguru import logger
import random
import yaml

# Initialize logger for tracking
logger.add("neurochess_log.log", rotation="10 MB")


1

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and selected.")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")
    import torch

if torch.cuda.is_available():
    print("PyTorch can see the GPU.")
else:
    print("PyTorch cannot see the GPU. Check CUDA installation.")


import torch
print(torch.cuda.is_available())


GPU is available and selected.
PyTorch can see the GPU.
True


In [3]:
!nvidia-smi
!nvcc --version

Fri Sep 27 22:32:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.09                 Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0             16W /   94W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# board.py
class BoardManager:
    def __init__(self):
        self.board = chess.Board()

    def make_move(self, move):
        """
        Make a move on the board (if valid).
        """
        try:
            chess_move = chess.Move.from_uci(move)
            if chess_move in self.board.legal_moves:
                self.board.push(chess_move)
            else:
                raise ValueError(f"Invalid move: {move}")
        except Exception as e:
            logger.error(e)

    def get_legal_moves(self):
        return list(self.board.legal_moves)

    def is_game_over(self):
        return self.board.is_game_over()

    def get_fen(self):
        return self.board.fen()

# Test the BoardManager
board_manager = BoardManager()
board_manager.make_move("e2e4")
print(board_manager.board)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [5]:
# mcts.py
class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def expand(self):
        for move in self.state.legal_moves:
            child_state = self.state.copy()
            child_state.push(move)
            child_node = MCTSNode(child_state, parent=self)
            self.children.append(child_node)

    def select_best_child(self):
        return max(self.children, key=lambda child: child.value / (child.visits + 1))

    def backup(self, value):
        self.value += value
        self.visits += 1
        if self.parent:
            self.parent.backup(value)

def monte_carlo_tree_search(root, num_simulations=100):
    for _ in range(num_simulations):
        node = root
        while node.children:
            node = node.select_best_child()
        node.expand()
        value = random.random()  # Simulate a random value for simplicity
        node.backup(value)

# Test MCTS
root_node = MCTSNode(board_manager.board)
monte_carlo_tree_search(root_node)
print(f"Root node visits: {root_node.visits}")


Root node visits: 100


In [6]:
# neural_network.py
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        self.fc1 = nn.Linear(773, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

    def fen_to_input(self, fen):
        # Convert FEN to a tensor input (simplified here)
        input_vector = [0] * 773
        return input_vector

# Test the neural network
model = ChessNet()
example_input = torch.rand(1, 773)
output = model(example_input)
print(f"Model output: {output.item()}")


Model output: 0.531063973903656


In [7]:
# game_manager.py
class GameManager:
    def __init__(self, player_white='human', player_black='ai', use_mcts=False):
        self.player_white = player_white
        self.player_black = player_black
        self.use_mcts = use_mcts
        self.board_manager = BoardManager()

    def play_game(self):
        current_player = 'white'
        while not self.board_manager.is_game_over():
            print(self.board_manager.board)
            if (current_player == 'white' and self.player_white == 'human') or \
               (current_player == 'black' and self.player_black == 'human'):
                move = input(f"{current_player}'s move: ")
                self.board_manager.make_move(move)
            else:
                if self.use_mcts:
                    root_node = MCTSNode(self.board_manager.board)
                    monte_carlo_tree_search(root_node)
                    best_move = root_node.select_best_child().state.peek()
                    self.board_manager.make_move(best_move.uci())
                else:
                    legal_moves = self.board_manager.get_legal_moves()
                    best_move = random.choice(legal_moves)
                    self.board_manager.make_move(best_move.uci())

            current_player = 'black' if current_player == 'white' else 'white'
        print("Game over!")
        print(self.board_manager.board.result())

# # Start a game (human vs AI with MCTS)
# game_manager = GameManager(player_white='human', player_black='ai', use_mcts=True)
# game_manager.play_game()


In [8]:
# rl_training.py
class RLTrainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        self.criterion = nn.MSELoss()
        # Check for GPU availability and move model to GPU
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
            self.model.to(self.device)
            print("Model moved to GPU.")
        else:
            self.device = torch.device("cpu")
            print("GPU not available, using CPU.")

    def train_step(self, board, reward):
        self.optimizer.zero_grad()
        # Changed: Call get_fen() to get the FEN string
        board_tensor = torch.tensor(self.model.fen_to_input(board.get_fen()), dtype=torch.float32).unsqueeze(0)

        # Move board_tensor to the device
        board_tensor = board_tensor.to(self.device)

        predicted_value = self.model(board_tensor)
        # Fix: Move target_value to the same device as predicted_value
        target_value = torch.tensor([reward], dtype=torch.float32).to(self.device)
        loss = self.criterion(predicted_value, target_value)
        loss.backward()
        self.optimizer.step()
        logger.info(f"Training step completed with loss: {loss.item()}")

    def run_training(self, num_games=100):
        """
        Runs the self-play and training loop for the specified number of games.

        Args:
            num_games (int): The number of self-play games to run for training.
        """
        for game_num in range(num_games):
            logger.info(f"Starting training game {game_num + 1}/{num_games}")

            # 1. Self-play a game
            game_manager = GameManager(player_white='ai', player_black='ai', use_mcts=True)
            game_data = []  # List to store (state, reward) pairs

            while not game_manager.board_manager.is_game_over():
                root_node = MCTSNode(game_manager.board_manager.board)
                monte_carlo_tree_search(root_node, num_simulations=100)  # Adjust num_simulations as needed
                best_move = root_node.select_best_child().state.peek()
                game_manager.board_manager.make_move(best_move.uci())

                # Store the current state and a temporary reward (0 for now)
                game_data.append(game_manager.board_manager.board.fen())

            # Assign final rewards based on game outcome
            game_result = game_manager.board_manager.board.result()
            if game_result == "1-0":  # White wins
                rewards = [1 if i % 2 == 0 else -1 for i in range(len(game_data))]
            elif game_result == "0-1":  # Black wins
                rewards = [-1 if i % 2 == 0 else 1 for i in range(len(game_data))]
            else:  # Draw
                rewards = [0] * len(game_data)

            # 2. Train the model on the game data
            for (state_fen, reward) in zip(game_data, rewards):
                board = BoardManager()  # Create a temporary board to load the state
                board.board.set_fen(state_fen)
                self.train_step(board, reward)

            logger.info(f"Training game {game_num + 1}/{num_games} completed.")

        logger.info("Training completed successfully.")
# Setup training
model = ChessNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer = RLTrainer(model, optimizer)

# Example training step
reward = 1.0
trainer.train_step(board_manager, reward)


Model moved to GPU.


C:\ProgramData\anaconda3\envs\env1\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
2024-09-27 22:33:10.571 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.23172686994075775


In [9]:
# RL Self-play and training loop
trainer.run_training(num_games=10)  # Train using 100 self-play games


2024-09-27 22:33:35.105 | INFO     | __main__:run_training:40 - Starting training game 1/10
2024-09-27 22:33:44.312 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.2718919813632965
2024-09-27 22:33:44.318 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.27161991596221924
2024-09-27 22:33:44.322 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.27036887407302856
2024-09-27 22:33:44.325 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.2686529755592346
2024-09-27 22:33:44.332 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.26666542887687683
2024-09-27 22:33:44.337 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.2644965648651123
2024-09-27 22:33:44.340 | INFO     | __main__:train_step:30 - Training step completed with loss: 0.26218923926353455
2024-09-27 22:33:44.346 | INFO     | __main__:train_step:30 - Training step completed with l

In [10]:
# prompt: now i want to save trained model in nnue file

import torch

# Assuming 'model' is your trained ChessNet model
# Save the model's state dictionary
torch.save(model.state_dict(), 'trained_model.pth')

# You will need to implement a conversion function to transform
# the PyTorch model weights into the NNUE format. This will involve
# understanding the NNUE file structure and extracting relevant weights
# from your model.

# Example (pseudocode):
# def convert_to_nnue(model_state_dict, output_file):
#   """Converts a PyTorch model to NNUE format."""
#   # Extract weights from the model state dictionary
#   # ...
#
#   # Write the weights to the output file in NNUE format
#   # ...

# Convert the model to NNUE and save it
# convert_to_nnue(model.state_dict(), 'trained_model.nnue')

print("Model saved successfully.")


Model saved successfully.


In [11]:
import torch
import numpy as np

def convert_to_nnue(model_state_dict, output_file):
    """Converts a PyTorch model to NNUE format."""

    # Extract weights and biases from the state dictionary
    fc1_weight = model_state_dict['fc1.weight'].numpy().transpose()
    fc1_bias = model_state_dict['fc1.bias'].numpy()
    fc2_weight = model_state_dict['fc2.weight'].numpy().transpose()
    fc2_bias = model_state_dict['fc2.bias'].numpy()
    fc3_weight = model_state_dict['fc3.weight'].numpy().transpose()
    fc3_bias = model_state_dict['fc3.bias'].numpy()

    # Create the NNUE data structure
    nnue_data = {
        'input_layer': {
            'weights': fc1_weight,
            'biases': fc1_bias
        },
        'hidden_layer_1': {
            'weights': fc2_weight,
            'biases': fc2_bias
        },
        'output_layer': {
            'weights': fc3_weight,
            'biases': fc3_bias
        }
    }

    # Save the NNUE data to a file
    with open(output_file, 'wb') as f:
        np.savez(f, **nnue_data)

# Load the PyTorch model
model = ChessNet()
model.load_state_dict(torch.load('trained_model.pth'))

# Convert and save the model in NNUE format
convert_to_nnue(model.state_dict(), 'trained_model.nnue')

print("Model converted and saved successfully.")

Model converted and saved successfully.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_12844\3605913704.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('trained_model.pth'))
